In [1]:
%load_ext autoreload
%autoreload 2

## Objective

Make a simple model to predict the number of issues, to determine what factors affect citizens contacting 311.

In [2]:
import os, sys
sys.path.append(os.path.join(os.path.dirname('.'), "../preprocessing"))

In [3]:
from transform_for_num_issues_pred import main as transform_dataset

In [4]:
from __future__ import division
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline

rcParams['figure.figsize'] = 20, 5
warnings.filterwarnings("ignore", category=DeprecationWarning)
sns.set_style("whitegrid")
sns.set_context("poster")

from helper_functions import dummify_cols_and_baselines, make_alphas

In [5]:
df_orig = pd.read_pickle('../data/data_from_remove_from_dataset.pkl')
df_orig.shape

(516406, 40)

In [6]:
df_transformed = transform_dataset(df_orig)
df_transformed.shape

../preprocessing/transform_for_num_issues_pred.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_subset['NUM_ISSUES'] = 1


True
> /home/ec2-user/311-prediction-times/preprocessing/transform_for_num_issues_pred.py(80)main()
-> return df
(Pdb) c


(822, 221)

## Choosing columns

In [9]:
cols_orig_dataset = ['NUM_ISSUES', 'SubmittedPhoto', 'Property_Type', 'Source', 'is_description']
cols_type = [col for col in df_transformed.columns if 'TYPE' in col]
cols_census = ['race_white',
     'race_black',
     'race_asian',
     'race_hispanic',
     'race_other',
     'poverty_pop_below_poverty_level',
     'earned_income_per_capita',
     'poverty_pop_w_public_assistance',
     'poverty_pop_w_food_stamps',
     'poverty_pop_w_ssi',
     'school',
     'school_std_dev',
     'housing',
     'housing_std_dev',
     'bedroom',
     'bedroom_std_dev',
     'value',
     'value_std_dev',
     'rent',
     'rent_std_dev',
     'income',
     'income_std_dev']

In [10]:
df_col_subset = df_transformed[cols_orig_dataset + cols_type + cols_census]
df_col_subset.shape

(822, 218)

In [11]:
df = df_col_subset.dropna().drop_duplicates()
df.shape

(550, 218)

## Dummify

In [12]:
cols_to_dummify = df.dtypes[df.dtypes == object].index
cols_to_dummify

Index([u'Property_Type', u'Source', u'school', u'housing'], dtype='object')

In [13]:
df_dummified, baseline_cols = dummify_cols_and_baselines(df, cols_to_dummify)

other is baseline 0 4
Constituent Call is baseline 1 4
8_6th_grade is baseline 2 4
rent is baseline 3 4


In [14]:
df_dummified.shape

(550, 228)

## Running model

In [72]:
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import make_scorer


In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified.drop('NUM_ISSUES', axis=1), 
    df_dummified.NUM_ISSUES, 
    test_size=0.2, 
    random_state=300
)

In [77]:
pipe = make_pipeline(StandardScaler(), LassoCV())

In [37]:
mse = make_scorer(mean_squared_error, greater_is_better=False)

In [75]:
params = {'lassocv__alphas': make_alphas(-10, 5)}
params = {'lassocv__alphas': [[1], [3], [10], [30], [100]]}
# params = {}
model = GridSearchCV(pipe, param_grid=params, n_jobs=-1, cv=10)
model.fit(X_train, y_train);

In [76]:
pd.DataFrame(model.cv_results_).T.iloc[:5]

,0,1,2,3,4
mean_fit_time,6.18584,7.58184,7.77761,7.56476,7.1074
mean_score_time,0.0288891,0.0238772,0.0233177,0.0189165,0.0111982
mean_test_score,-0.075384,0.184089,0.359286,0.414184,0.403536
mean_train_score,0.985118,0.984363,0.982475,0.978525,0.969958
param_ridgecv__alphas,[1],[3],[10],[30],[100]


In [71]:
model.score(X_test, y_test)

0.55862806412569355

## Conclusion

Even with Lasso regularization, the model overfits to the training set and underfits the test set. We are still in a high-variance situation. I could use dimensionality reduction to mitigate this, but that would impact the interpretability of my model.

Since my model would be less interpretable if I want it to perform better, I will use Random Forests to hopefully decrease the variance.

## For future work

- Incorporate time series aspect